In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
import os
os.getcwd()

'C:\\Users\\akadali\\Desktop\\Deep_NLP\\MLG_Capstone_ChatBot\\ChatBot_GoogleW2V'

In [3]:
os.chdir('C:\\Users\\akadali\\Desktop\\Deep_NLP\\MLG_Capstone_ChatBot\\ChatBot_GoogleW2V\\raw_data')

In [4]:
data = pd.read_csv('Referral Emails.CSV', encoding = 'latin1')

Email Data Cleanup Steps
-----------
Below steps are performed on EMail dataset to preprocess and extract body texts from the email. This dataset is acquired by exporting emails from the last 3 years into a 'csv' file.
* Drop the columns which are of no use
* Drop unnecessary columns
* Remove Emails with empty body text - empty subject
* Remove Automatic replies
* Remove undeliverable emails
* Remove all 'na' values
* Remove emails with Subject has 'Validation' and 'Yammer'
* Remove LCSP Approval emails
* Remove Emails with Subjet has 'Bot' in it...as these are all bot status emails and are of no use for us
* Remove Referral status Communications - These are the communication sent out to employees and in return, they ask questions. For now these can be removed as we are looking for first contact queries
* Looking for most frequent Emails subjects. If a particular subject is appearing too many times, probably that consists of long chain of emails and fo no good to our cause. Hence remove the emails with that subject
* Remove Emails sent from US Talent Referral Program and 'Replied Emails'. As these are the response sent by the Talent Referral team and we do not need them for now, as we are trying to work on questions here.
* Dropping emails with Subjects start with 'RE' and 'Re' as we need only FCQs (First Contact Queries)
* Check if there are any emails with subject 'Action Required'. These are the validation emails sent out to recruiters by the team
* Look at the Most frequent statuses again adn remove if there are any
* Removing all rows/records that are not sent to US Talent Referral Program mailbox
* Removing emails received from "Callahan, Jane", "Tiffany, Mercer", "US CTS Talent R&A", 'Deloitte (O365D) on Yammer'm "YourOnlineProfessor", "US Talent CIC Inbox", "US R10 Referral Response"
* Drop the remaining unnecessary columns
* Removing Signatures from the emails
* Calculating 'Verbs count', 'Line Count', 'Total Word Count' and 'Question Length' for further analysis

# Data Cleanup

* Drop the columns which are of no use

In [5]:
data.drop(['BCC: (Name)', 'BCC: (Address)', 'BCC: (Type)', 'Billing Information', 'Categories', 'Importance', 'Mileage', 'Sensitivity'], axis = 1, inplace = True)

In [6]:
data.shape

(27872, 11)

* This dataset has about 27872 emails

In [7]:
#A definition to look at the quick descriptive statistics of the dataset
def quick_analysis(data):
    analysis = pd.DataFrame({"Data Types": data.dtypes, 
                             "Null Values": data.apply(lambda x:sum(x.isnull())),
                             "NaN Values": data.isna().sum(),
                             "Count": data.count(),
                             #"Mean": data.mean().round(2),
                             #"First Quantile":data.quantile(0.25).round(2),
                             #"Median or Q2":data.median().round(2),
                             #"Thrid Quantile":data.quantile(0.75).round(2),
                             #"IQR":(data.quantile(0.75)-data.quantile(0.25)).round(2),
                             #"No. of outliers":data[((data > data.quantile(0.75) + 1.5*((data.quantile(0.75)-data.quantile(0.25)).round(2))) | (data < data.quantile(0.25).round(2)-1.5*(data.quantile(0.75)-data.quantile(0.25)).round(2)))].count(),
                             #"Outliers above UQR":data[(data > data.quantile(0.75) + 1.5*(data.quantile(0.75)-data.quantile(0.25)).round(2))].count(),
                             "Unique Count": data.nunique()})
    return analysis

* Drop unnecessary columns

In [8]:
data.drop(['CC: (Name)','CC: (Address)', 'CC: (Type)'], axis = 1, inplace = True)

In [9]:
#quick_analysis(data)

* Remove Emails with empty body text - empty subject

In [10]:
data.dropna(axis = 0)

,Subject,Body,From: (Name),From: (Address),From: (Type),To: (Name),To: (Address),To: (Type)
0,"Automatic reply: When we have great talent, yo...","Hello,\r\n\r\n \r\n\r\nThank you for your mess...","Blanchard, John",/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,EX,US Talent Referral Program,/o=ExchangeLabs/ou=Exchange Administrative Gro...,EX
1,"Automatic reply: When we have great talent, yo...",**Automatic reply**\r\n\r\n \r\n\r\nDear Sende...,"Rocolcol, Lady Rose",/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,EX,US Talent Referral Program,/o=ExchangeLabs/ou=Exchange Administrative Gro...,EX
2,"Automatic reply: When we have great talent, yo...","Hi, I am currently out of the office and will ...","Altman, Bradley",/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,EX,US Talent Referral Program,/o=ExchangeLabs/ou=Exchange Administrative Gro...,EX
3,"Automatic reply: When we have great talent, yo...","I am currently out of the office, returning Tu...","Fonseca, Mary Clare",/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,EX,US Talent Referral Program,/o=ExchangeLabs/ou=Exchange Administrative Gro...,EX
4,"Automatic reply: When we have great talent, yo...",Thank you for your email. I will be out of the...,"Vasquez, Aimee",/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,EX,US Talent Referral Program,/o=ExchangeLabs/ou=Exchange Administrative Gro...,EX
...,...,...,...,...,...,...,...,...
27867,RE: Referral Submission Resume Issue -10111410,"Hi Christopher,\r\n\r\n \r\n\r\nThank you for ...",US Talent Referral Program,/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,EX,"Hamilton, Christopher",/o=ExchangeLabs/ou=Exchange Administrative Gro...,EX
27868,RE: Referral Source Question 25888696,"Hi Aaron,\r\n\r\n \r\n\r\nHope you are doing w...",US Talent Referral Program,/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,EX,"Fleischmann, Aaron",/o=ExchangeLabs/ou=Exchange Administrative Gro...,EX
27869,RE: Referral Source Question 25888696,"Hi Angela,\r\n\r\n \r\n\r\nThank you for the u...",US Talent Referral Program,/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,EX,"Goss, Angela",/o=ExchangeLabs/ou=Exchange Administrative Gro...,EX
27870,RE: Non-Target School Campus Recruiting,"Hi Lucas,\r\n\r\n \r\n\r\nThank you for reachi...",US Talent Referral Program,/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,EX,"Booth, Lucas",/o=ExchangeLabs/ou=Exchange Administrative Gro...,EX


* Remove Automatic replies

In [11]:
#type(data['Subject'].str.contains('Automatic reply'))
#data[data['Subject'].str.contains('Automatic reply', na = False)]

In [12]:
data.drop(data[data['Subject'].str.contains('Automatic reply', na = False)].index, inplace = True)

* Remove undeliverable emails

In [13]:
#data['Subject'].str.contains("Undeliverable").sum()

In [14]:
data.drop(data[data['Subject'].str.contains("Undeliverable", na = False)].index, inplace = True)

* Remove all na values

In [15]:
data.dropna(axis = 0, inplace = True)

* Remove emails with Subject has 'Validation' and 'Yammer'

In [16]:
#data['Subject'].str.contains("Validation").sum()
data.drop(data[data['Subject'].str.contains("Validation", na = False)].index, inplace = True)
data.drop(data[data['Subject'].str.contains("Yammer", na = False)].index, inplace = True)
data.drop(data[data['Subject'].str.contains("validation", na = False)].index, inplace = True)

* Remove LCSP Approval emails

In [17]:
#data['Subject'].str.contains("LCSP").sum()
data.drop(data[data['Subject'].str.contains("LCSP", na = False)].index, inplace = True)

* Remove Emails with Subjet has 'Bot' in it...as these are all bot status emails and are of no use for us

In [18]:
#data['Subject'].str.contains("Bot").sum()
data.drop(data[data['Subject'].str.contains("Bot", na = False)].index, inplace = True)

* Remove Referral status Communications - These are the communication sent out to employees and in return, they ask questions. For now these can be removed as we are looking for first contact queries

In [19]:
print(data['Subject'].str.contains("Referral Status").sum())
print(data['Subject'].str.contains("Status of your referral").sum())
print(data['Subject'].str.contains("referral status").sum())
print(data['Subject'].str.contains("Status of your Referral").sum())

994
8
14
2604


In [20]:
data.drop(data[data['Subject'].str.contains("Referral Status", na = False)].index, inplace = True)
data.drop(data[data['Subject'].str.contains("Status of your referral", na = False)].index, inplace = True)
data.drop(data[data['Subject'].str.contains("referral status", na = False)].index, inplace = True)
data.drop(data[data['Subject'].str.contains("Status of your Referral", na = False)].index, inplace = True)

* Looking for most frequent Emails subjects. If a particular subject is appearing too many times, probably that consists of long chain of emails and fo no good to our cause. Hence remove the emails with that subject

In [21]:
#data['Subject'].value_counts(sort = True)[:20]
#data['Subject'].value_counts(sort = True)[:20].sum()
subs = pd.Series(data['Subject'].value_counts(sort = True)[:20])
subs = subs.index.tolist()

In [22]:
#writing a method to remove most frequent subjects from the database
def remove_sub(sub):
    data.drop(data[data['Subject'].str.contains(sub, na = False)].index, inplace = True)

In [23]:
#Removing a few subjects, as we may lose more information 
rem_list = ['RE: Referral Bonus','RE: Referral','RE: Referral Question','Referral','Referral Question']
for sub in rem_list:
    subs.remove(str(sub))

In [24]:
for sub in subs:
    remove_sub(str(sub))

C:\Users\akadali\Anaconda\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [25]:
#now if we check the size of a data frame...991 records should have been removed.
#data.shape

* Remove Emails sent from US Talent Referral Program and 'Replied Emails'. As these are the response sent by the Talent Referral team and we do not need them for now, as we are trying to work on questions here.

In [26]:
#data['From: (Name)'].str.contains('US Talent Referral Program').sum()
data.drop(data[data['From: (Name)'].str.contains('US Talent Referral Program', na = False)].index,inplace = True)

* Dropping emails with Subjects start with 'RE' and 'Re' as we need only FCQs (First Contact Queries)

In [27]:
#data['Subject'].str.startswith("Re:").sum()
#data['Subject'].str.startswith("RE:").sum()
data.drop(data[data['Subject'].str.startswith("RE:", na = False)].index, inplace = True)
data.drop(data[data['Subject'].str.startswith("Re:", na = False)].index, inplace = True)

* Check if there are any emails with subject 'Action Required'. These are the validation emails sent out to recruiters by the team

In [28]:
data.drop(data[data['Subject'].str.contains("Action Required", na = False)].index, inplace = True)
data.drop(data[data['Subject'].str.contains("ACTION REQUIRED", na = False)].index, inplace = True)

* Look at the Most frequent statuses again adn remove if there are any

In [29]:
#data['Subject'].value_counts()[:20]

In [30]:
data.drop(data[data['Subject'].str.startswith("FW: [EXT] Thank you for your referral to Deloitte", na = False)].index, inplace = True)

In [31]:
data['To: (Name)'].value_counts()

US Talent Referral Program                                                948
US Talent Referral Program (US - Stamford)                                863
PPSS Field II                                                              35
Agarwal, Jaideep                                                           13
Mercer, Tiffany;Callahan, Jane                                             12
                                                                         ... 
Loreto, Evian (US - Denver);Breck, Kim (US - Minneapolis)                   1
US Talent Referral Program (US - Stamford);Loreto, Evian (US - Denver)      1
Garren, Kacee (US - Arlington)                                              1
Singhal, Prashant                                                           1
Johansen, Alex                                                              1
Name: To: (Name), Length: 188, dtype: int64

* Removing all rows/records that are not sent to US Talent Referral Program mailbox

In [32]:
#Removing all rows/records that are not sent to US Talent Referral Program mailbox
data.drop(data[~data['To: (Name)'].str.contains("US Talent Referral Program", na = False)].index, inplace = True)

* Removing emails received from "Callahan, Jane", "Tiffany, Mercer", "US CTS Talent R&A", 'Deloitte (O365D) on Yammer'm "YourOnlineProfessor", "US Talent CIC Inbox", "US R10 Referral Response"

In [33]:
fname_list2 = ["Callahan, Jane", "Tiffany, Mercer", "US CTS Talent R&A", 
             "Deloitte (O365D) on Yammer ", "YourOnlineProfessor", 
             "US Talent CIC Inbox", "US R10 Referral Response", "Deloitte Global Recruiting", "Yammer"]

In [34]:
def remove_from(fname):
    data.drop(data[data['From: (Name)'].str.contains(fname, na = False)].index, inplace = True)

for fname in fname_list2:
    remove_from(fname)

* Remove unecessary variables from the remaining columns

In [35]:
data.drop(['From: (Name)', 'From: (Address)', 'From: (Type)', 'To: (Name)', 'To: (Address)', 'To: (Type)'], 
          axis = 1, inplace = True)

# Removing Signatures from the emails

Signatures for all emails should be removed as we are going to need only the emails body texts for our training data. Removing/separating signatures from the actual email bodies involves below steps

* Breakdown email body into paragraphs

* Calculate POS of all words using nltk

* Any paragraph that doesn't have enough/no verbs is a signature and that should be trimmed off

* Verb count for each paragraph/sentence is calculated, the second para/line where we have no verbs, that's where our email body ends and signature starts

* In summary, it discards blocks that do not contain enough verbs to be considered a message block, being treated as signature blocks instead.

* Information source: https://kinoshita.eti.br/2017/06/14/how-to-remove-the-signature-from-emails-with-nlp.html

* Below function exactly does that

In [38]:
from nltk.tokenize import blankline_tokenize
from collections import Counter
def trim_signature(email):
    paras = blankline_tokenize(str(email)) #divdes the email into small paragraphs
    v_count = []                           #List of verbs count for each line
    lines = []                             #List of lines
    line_count = 0                         #Number of Lines
    lv_count = 0                           #Lines with verbs
    nv_count = 0                           #Lines with no-verbs
    for line in paras:
        line_count += 1
        tokens = nltk.word_tokenize(line.lower())  
        text = nltk.Text(tokens)
        tags = nltk.pos_tag(text)
        counts = Counter(tag for word,tag in tags)
        verbs = counts['VB']+counts['VBD']+counts['VBG']+counts['VBN']+counts['VBP']+counts['VBZ']
        if verbs != 0 :
            lv_count += 1
            lines.append(line)
            v_count.append(verbs)
        else:
            nv_count += 1
        if nv_count == 2:
            break
    return " ".join(lines)

In [53]:
from nltk.tokenize import blankline_tokenize
from collections import Counter
def trim_signatures2(email):
    paras = blankline_tokenize(str(email)) #divdes the email into small paragraphs
    v_count = []                           #List of verbs count for each line
    lines = []                             #List of lines
    line_count = 0                         #Number of Lines
    lv_count = 0                           #Lines with verbs
    nv_count = 0                           #Lines with no-verbs
    for line in paras:
        line_count += 1
        tokens = nltk.word_tokenize(line.lower())  
        text = nltk.Text(tokens)
        tags = nltk.pos_tag(text)
        counts = Counter(tag for word,tag in tags)
        verbs = counts['VB']+counts['VBD']+counts['VBG']+counts['VBN']+counts['VBP']+counts['VBZ']
        print("Sentence#",line_count)
        print("No. of Verbs", verbs)
    return paras

In [54]:
k = trim_signatures2(data['Body'].iloc[123])

Sentence# 1
No. of Verbs 0
Sentence# 2
No. of Verbs 2
Sentence# 3
No. of Verbs 0
Sentence# 4
No. of Verbs 2
Sentence# 5
No. of Verbs 0
Sentence# 6
No. of Verbs 5
Sentence# 7
No. of Verbs 1
Sentence# 8
No. of Verbs 0
Sentence# 9
No. of Verbs 2
Sentence# 10
No. of Verbs 0
Sentence# 11
No. of Verbs 1
Sentence# 12
No. of Verbs 0
Sentence# 13
No. of Verbs 1
Sentence# 14
No. of Verbs 0
Sentence# 15
No. of Verbs 4
Sentence# 16
No. of Verbs 1
Sentence# 17
No. of Verbs 0
Sentence# 18
No. of Verbs 0
Sentence# 19
No. of Verbs 1
Sentence# 20
No. of Verbs 1


In [55]:
k

['Hi there,',
 'Can you let me know the referral bonus for an international tax manager for the OC / LA office?',
 'Thank you',
 'From: Chavira-Aubel, Sarah (US - San Diego) \r\nSent: Friday, June 7, 2019 3:27 PM\r\nTo: Chaudhry, Ausaff Arshad (US - San Jose) <achaudhry@deloitte.com>\r\nSubject: RE: Referral bonus',
 'Hi Ausaff,',
 'Please email USTalentReferralProgram@deloitte.com <mailto:USTalentReferralProgram@deloitte.com>  and they can let you know when it will be paid out.',
 'Thank you and have a great weekend!',
 'Sarah Chavira-Aubel | Deloitte\x95',
 'Lead Experienced Recruiting Specialist | Tax',
 'Direct: +1 619-237-6728 | Mobile: +1 858-281-9314',
 'schaviraaubel@deloitte.com <mailto:schaviraaubel@deloitte.com>  | www.deloitte.com <http://www.deloitte.com/>',
 'Explore Deloitte Tax LLP <https://usrecruiting.deloitte.com/experienced-hire-tax-talent-information>',
 'From: Chaudhry, Ausaff Arshad (US - San Jose) \r\nSent: Thursday, June 6, 2019 3:30 PM\r\nTo: Chavira-Aubel, Sa

In [37]:
#Applying the above method
data['email_cleaned'] = data['Body'].apply(lambda x: trim_signature(x))

* Email body texts have strange characters like X97,\X94,\X93. They should be removed

In [38]:
import re
def text_clean1(text):
    text = re.sub("[\x97]","'", text)
    text = re.sub("[\x96]","'", text)
    text = re.sub("[\x95]","'", text)
    text = re.sub("[\x94]","'", text)
    text = re.sub("[\x93]","'", text)
    text = re.sub("[\x92]","'", text)
    text = re.sub("[\x91]","'", text)
    return text

#apply the function
data['email_cleaned'] = data['email_cleaned'].apply(lambda x: text_clean1(x))

In [39]:
#Sample test
data['email_cleaned'].iloc[87]

"As part of the BTA Core School Referral for Spring 2019, I referred Niki Zhao <https://people.deloitteresources.com/dpn/index.html#/Profile/nikzhao>  (zhao13@iu.edu <mailto:zhao13@iu.edu> ) from Indiana University in Fall 2018. She was offered the position and joined the firm in Feb 2019. I haven't received any communication from the Referral program ever since and wanted to follow up with you regarding this. I reached out to Delaine McAndrews <mailto:demcandrews@DELOITTE.com>  (IU School Recruiter) in order to refer Niki and was told that she was selected to interview for round 1. Do you happen to have any records of the same?"

* Calculating the number of words and characters for every email...after they are cleaned

In [40]:
#Creating a column of with word counts and char counts in each sentence
data['tw_pre_cleaning'] = data['Body'].str.split().str.len()
data['tc_pre_cleaning'] = data['Body'].str.strip().str.len()
data['tw_post_cleaning'] = data['email_cleaned'].str.split().str.len()
data['tc_post_cleaning'] = data['email_cleaned'].str.strip().str.len()

In [41]:
#Dropping the unnecesary columns
#data.drop(['totalwords', 'totalchars'], axis = 1, inplace = True)

In [48]:
print("Average no. of words in an email before cleaning the data", np.round(data['tw_pre_cleaning'].mean(),2))
print("Average no. of words in an email after cleaning the data", np.round(data['tw_post_cleaning'].mean(), 2))
print("Difference or unnecessary words removed", np.round(data['tw_pre_cleaning'].mean()-data['tw_post_cleaning'].mean(),2))

Average no. of words in an email before cleaning the data 197.87
Average no. of words in an email after cleaning the data 52.57
Difference or unnecessary words removed 145.3


In [43]:
#Defeinitions to calculate Verbs and lines in all email inquiries
def ver_counter(email):
    paras_ = blankline_tokenize(str(email)) #divdes the email into small paragraphs
    verbs_ = 0
    for line in paras_:
        tokens = nltk.word_tokenize(line.lower())  
        text = nltk.Text(tokens)
        tags = nltk.pos_tag(text)
        counts = Counter(tag for word,tag in tags)
        verbs_ = counts['VB']+counts['VBD']+counts['VBG']+counts['VBN']+counts['VBP']+counts['VBZ']
    return verbs_


def line_counter(email):
    paras_ = blankline_tokenize(str(email)) #divdes the email into small paragraphs
    return len(paras_)

In [44]:
data['verbs'] = data['email_cleaned'].apply(lambda x:ver_counter(x))
data['lines'] = data['email_cleaned'].apply(lambda x:line_counter(x))

In [49]:
data['verbs'].mean()

11.577136514983351

In [45]:
#Resetting the index and dropping the column
data = data.reset_index()
data.drop(['index'], axis = 1, inplace = True)

# Data Backup #1

In [46]:
data_backup1 = data
data_backup1.to_csv('C:\\Users\\akadali\\Desktop\\Deep_NLP\\MLG_Capstone_ChatBot\\ChatBot_GoogleW2V\\data_Preprocessed.csv', 
                    index = False, header = True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
sns.distplot(data["tw_post_cleaning"], bins=30)
plt.title("Word count Distribution across emails", fontsize = 15)
plt.axvline(210)
plt.xticks(np.arange(0,500,30))
plt.show()

In [ ]:
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(20,6)})
sns.boxplot(x=data['tw_post_cleaning'], color = 'm')
#plt.axvline(36,0, linestyle = "--", color = 'red')
plt.title("Word count Distribution across emails", fontsize = 15)
plt.xticks(np.arange(0,450,30))
plt.show()

* From the above it is evident that emails with more than 120 words are outliers
* These emails to be removed as we are going to need only those emails with minimal text that can be given as input questions to the bot

In [ ]:
#Dropping the records with more than 120 words - There are 80 such rferral queries with more than 80 words
data.drop(data[data['tw_post_cleaning']>120].index, inplace = True)

Word Count Distributions post removing the emails with large texts

In [ ]:
sns.distplot(data["tw_post_cleaning"], bins=30)
plt.title("Word count Distribution across emails", fontsize = 15)
#plt.xticks(np.arange(0,500,30))
plt.show()

In [ ]:
sns.distplot(data["tc_post_cleaning"], bins = 200)
plt.title("Question Length", fontsize = 20)
#plt.xticks(np.arange(0,500,30))
plt.show()

In [ ]:
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(20,6)})
sns.boxplot(x=data['tw_post_cleaning'], color = 'm')
#plt.axvline(36,0, linestyle = "--", color = 'red')
plt.title("Word count Distribution across emails", fontsize = 15)
plt.xticks(np.arange(0,120,10))
plt.show()

* Majority of emails have text lengths between 30 and 60 words.

In [ ]:
sns.distplot(data["verbs"], bins=30)
plt.title("Verbs Distribution across emails", fontsize = 15)
#plt.xticks(np.arange(0,500,30))
plt.show()

# Data Backup #2

In [ ]:
#Creating a back-up
#data_step1 = data
#Exporting to a dataframe
data_step1.to_csv('C:\\Users\\akadali\\Desktop\\Deep_NLP\\MLG_Capstone_ChatBot\\ChatBot_GoogleW2V\\data_Preprocessed2.csv', index = False, header = True)

In [ ]:
data_step1.head()

Text Mining/Text Analytics
-------

Data Clean-up
**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Remove contractions
* Remove stop words
* Tokenize text

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more

In [ ]:
#Converting data to lower case
col = 'email_cleaned'
data[col] = data[col].apply(lambda x: x.lower())

#Removing Punctuations
puncs = string.punctuation
puncs = puncs + "’'`"
data[col] = data[col].apply(lambda x: re.sub('[%s]'%re.escape(puncs), ' ', x))

#Removing Stop Words
stops = nltk.corpus.stopwords.words('english')
data[col] = data[col].apply(lambda x: " ".join(w for w in x.split() if w not in stops))

In [ ]:
#The text data has a lot of contractions and all of them need to be cleared to 
#avoid any unsual characters after preprocessing the text
def text_clean(text):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\r", " ", text)
    return text

#Applying the above function
data[col] = data[col].apply(lambda x: text_clean(x))

In [ ]:
#We still have some extra spaces present in the data. Let’s remove them
#Removing extra spaces that might have been created while removing punctuations
data[col] = data[col].apply(lambda x: re.sub('  +',' ',x))

In [ ]:
#tokenizing the text

from nltk.tokenize import word_tokenize
corpus = " ".join(data[col])
tokens = word_tokenize(corpus)
print("Total number of words in the corpus - ", len(tokens))

In [ ]:
#Function to get the unique list of words
def unique_list(text):
    unique_list = []
    for w in text.split():
        if w not in unique_list:
            unique_list.append(w)
    return unique_list

#Getting Lemmas for corpus tokens
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
lems = " ".join(lemma.lemmatize(w, pos = 'v') for w in tokens)
unique_lems = unique_list(lems)
print("Number of unique (root) words in corpus - ", len(unique_lems))

In [ ]:
#Finding word frequencies of all words in the corpus tokens
from nltk.probability import FreqDist
fdist = FreqDist()

for word in tokens:
    fdist[word] += 1
    

#Top 20 tokens with highest frequency 
fdist_top20 = fdist.most_common(20)
word_freq = pd.DataFrame(fdist_top20)
word_freq.columns = ['word','freq']

In [ ]:
#Bar Chart to show the top 20 most frequent words
plt.rcParams['figure.figsize'] = [10, 6]
plt.barh(word_freq['word'],word_freq['freq'])
plt.xlabel("Top_Words")
plt.ylabel("Count")
plt.title("Top Words Frequency", fontsize = 15)
plt.show()

In [ ]:
#Let's Generate a wordcloud to observer the highligted words over the entire corpus
from wordcloud import WordCloud
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stops, 
                min_font_size = 10).generate(corpus)

# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.title("Unigram Wordcloud")
plt.tight_layout(pad = 0) 
plt.show() 

In [ ]:
#Lets get the bigrams and trigrams
from nltk import ngrams
bigrams = list(ngrams(corpus.split(), 2))
trigrams = list(ngrams(corpus.split(), 3))

bigrams = pd.DataFrame(bigrams)
bigrams = bigrams[0] + "_" + bigrams[1] 

bigram_words = ' '
for val in bigrams:
    val = str(val)
    tokens = val.split()
    bigram_words += " ".join(tokens)+" "
    
#Top 20 bigrams
bigrams.value_counts()[:20]

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stops, 
                min_font_size = 10).generate(bigram_words) 

# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.title("Bigram Word Cloud", fontsize = 15)
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
#Trigrams word clouds
trigrams = pd.DataFrame(trigrams)
trigrams = trigrams[0]+'_'+trigrams[1]+'_'+trigrams[2]
trigrams.value_counts()[:20]

In [ ]:
trigram_words = ' '
for val in trigrams:
    val = str(val)
    tokens = val.split()
    trigram_words += " ".join(tokens)+" "
    
bigram_words
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stops, 
                min_font_size = 10).generate(trigram_words) 

# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.title("Trigrams Wordcloud", fontsize = 15)
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show() 

# Let's try LDA model to determine the topics in the dataset

In [ ]:
#Let's attempt to use CountVectorizer document-term matrix and gensim models for topic modelling

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words = 'english')
data_cv = cv.fit_transform(data[col])
data_dtm = pd.DataFrame(data_cv.toarray(), columns = cv.get_feature_names())
#data_dtm.index = qa_data.index
#data_dtm

In [ ]:
from gensim import matutils, models
import scipy.sparse

# One of the required inputs is a term-document matrix
tdm = data_dtm.transpose()
tdm.head()

# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
#cv.vocabulary_
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [ ]:
#Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term)

#LDA for num_topics = 2
lda_2 = models.LdaModel(corpus = corpus, id2word = id2word, num_topics = 2, passes = 10)
lda_2.print_topics()

In [ ]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus = corpus, id2word = id2word, num_topics = 3, passes = 10)
lda.print_topics()

In [ ]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus = corpus, id2word = id2word, num_topics = 4, passes = 10)
lda.print_topics()

In [ ]:
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

data_nouns_adj = pd.Series(data[col].apply(nouns_adj))
#data_nouns_adj.shape
data_nouns_adj.head()

In [ ]:
"""data_nouns_adj"""
"""LDA with Nouns and Adjectives"""
cvna = CountVectorizer(stop_words = 'english')
dat_cvna = cvna.fit_transform(data_nouns_adj)
data_dtna = pd.DataFrame(dat_cvna.toarray(), columns = cvna.get_feature_names())

# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
"""LDA with nouns & adjectives - no. of topics = 2"""
ldna = models.LdaModel(corpus = corpusna, id2word = id2wordna, num_topics = 2, passes = 10)
ldna.print_topics()

In [ ]:
"""LDA with nouns & adjectives - no. of topics = 3"""
ldna_3 = models.LdaModel(corpus = corpusna, id2word = id2wordna, num_topics = 3, passes = 10)
ldna_3.print_topics()

* Topic#1 - Job positions, resume submissions
* Topic#2 - Referral bonus
* Topic#3 - 

In [ ]:
"""LDA with nouns & adjectives - no. of topics = 4"""
ldna_4 = models.LdaModel(corpus = corpusna, id2word = id2wordna, num_topics = 4, passes = 10)
ldna_4.print_topics()

In [ ]:
"""LDA with nouns & adjectives - no. of topics = 8"""
ldna_8 = models.LdaModel(corpus = corpusna, id2word = id2wordna, num_topics = 8, passes = 10)
ldna_8.print_topics()